In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import keras_preprocessing
from keras_preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
train_path = '/Users/28634/Desktop/HKU/Computer_Science/exchange_CS3237/group_project/fruits/train'
test_path = '/Users/28634/Desktop/HKU/Computer_Science/exchange_CS3237/group_project/fruits/test'

In [4]:
train_gen = train_datagen.flow_from_directory(train_path,
                                              target_size = (150,150),
                                              class_mode = "categorical")
test_gen = test_datagen.flow_from_directory(test_path,
                                            target_size = (150,150),
                                            class_mode = "categorical")

Found 14401 images belonging to 6 classes.
Found 3198 images belonging to 6 classes.


In [5]:
train_freshapples_path = r"C:\Users\28634\Desktop\HKU\Computer_Science\exchange_CS3237\group_project\fruits\train\freshapples"
number_freshapples_train = len(os.listdir(train_freshapples_path))
print("total training freshapple images: ", number_freshapples_train)

train_rottenapples_path = r"C:\Users\28634\Desktop\HKU\Computer_Science\exchange_CS3237\group_project\fruits\train\rottenapples"
number_rottenapples_train = len(os.listdir(train_rottenapples_path))
print("total training rottenapple images: ", number_rottenapples_train)


total training freshapple images:  1693
total training rottenapple images:  2342


In [13]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(150, 150, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='softmax'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 150, 150, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 75, 75, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 73, 73, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

In [15]:
model.compile(loss = "categorical_crossentropy", optimizer = "rmsprop", metrics = ["accuracy"])

In [19]:
test_accuracy = 0.995

class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        acc = logs.get("test_acc")
        if((acc != None) and (acc >= test_accuracy)):
            print("\nReached desired validation accuracy, so cancelling training")
            self.model.stop_training = True
        
callbacks = myCallback()


In [ ]:
fruit_model = model.fit(train_gen, epochs = 100, validation_data = test_gen, verbose = 1, callbacks = [callbacks], workers = 10)
filepath = r"C:\Users\28634\Desktop\HKU\Computer_Science\exchange_CS3237\group_project\model_part1"
save_model(
    model,
    filepath,
    overwrite = True,
    include_optimizer = True,
    save_format = "tf",
    signatures = None
)

model.save("fruit_classification.h5")

Epoch 1/100
451/451 [==============================] - 589s 1s/step - loss: 0.3318 - accuracy: 0.8957 - val_loss: 0.2839 - val_accuracy: 0.8984
Epoch 2/100
451/451 [==============================] - 637s 1s/step - loss: 0.2130 - accuracy: 0.9335 - val_loss: 0.5905 - val_accuracy: 0.8468
Epoch 3/100
451/451 [==============================] - 615s 1s/step - loss: 0.1572 - accuracy: 0.9534 - val_loss: 0.2017 - val_accuracy: 0.9321
Epoch 4/100
451/451 [==============================] - 614s 1s/step - loss: 0.1354 - accuracy: 0.9642 - val_loss: 0.2206 - val_accuracy: 0.9443
Epoch 5/100
451/451 [==============================] - 622s 1s/step - loss: 0.1349 - accuracy: 0.9686 - val_loss: 0.2283 - val_accuracy: 0.9487
Epoch 6/100
451/451 [==============================] - 554s 1s/step - loss: 0.1293 - accuracy: 0.9703 - val_loss: 0.1376 - val_accuracy: 0.9659
Epoch 7/100
451/451 [==============================] - 967s 2s/step - loss: 0.1166 - accuracy: 0.9736 - val_loss: 0.3774 - val_accuracy:

In [ ]:
acc = fruit_model.history['acc']
val_acc = fruit_model.history['val_acc']
loss = fruit_model.history['loss']
val_loss = fruit_model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = "Training Accuracy")
plt.plot(epochs, val_acc, 'b', label = "Validation Accuracy")
plt.title("Training and Validation Accuracy Scores")
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'ro', label = "Training Loss")
plt.plot(epochs, val_loss, 'r', label = "Validation Loss")
plt.title("Training and Validation Loss Scores")
plt.legend()
plt.show()